In [2]:
import h5py
import tables # Required h5py to work properly here
import numpy as np
from pathlib import Path
import pandas as pd
import xarray as xr
import yaml

import logging
logger = logging.getLogger(__name__)

In [ ]:
if __name__ == "__main__":

    region_fn = snakemake.input["region_definitions"]

    with open(region_fn, "r") as f:
        regions = yaml.safe_load(f)

    region = regions["regions"][snakemake.wildcards.region]

    region = sorted(region) # Do this also in region creation (in GEGIS) to ensure same order

    era_year = int(snakemake.wildcards["era_year"])
    ssp_year = int(snakemake.wildcards["ssp_year"])
    ssp_scenario = snakemake.wildcards["ssp"]
    rcp_scenario = snakemake.wildcards["rcp"]
    
    with h5py.File(snakemake.input["demand"], "r") as f:
        demand = np.array(f["demand"])

    # Convert to xarray and add appropriate dimensions+coordinates
    demand = xr.DataArray(demand,
                    name="Electricity demand",
                    dims=["region", "time"],
                    coords={
                        "region": region,
                        "time":   pd.date_range(start=str(era_year),
                                                end=str(era_year+1),
                                                freq="H",
                                                closed="left"
                                               )
                    },
                    attrs={
                        "unit":"MWh",
                        "energy carrier":"electricity",
                        "description":"Projected electricity demand using GlobalEnergyGIS",
                        "projection year": ssp_year,
                        "weather data year (ERA5)": era_year,
                        "SSP scenario": f"{ssp_scenario}",
                        "RCP scenario": f"{rcp_scenario}",
                    }
                )

    demand.to_netcdf(snakemake.output["netcdf"])
    demand.to_dataframe().to_csv(snakemake.output["csv"], sep=";", quotechar="\"")